In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
from matplotlib.colors import LinearSegmentedColormap
from highlight_text import HighlightText, ax_text, fig_text

In [92]:
the_analyst_players = pd.read_csv('the_analyst_players.csv')
the_analyst_teams = pd.read_csv('the_analyst_team.csv')
fbref_teams = pd.read_csv('squad_indiv.csv').apply(pd.to_numeric,errors='ignore')
fbref_players = pd.read_csv('player_stats.csv').apply(pd.to_numeric,errors='ignore')
fbref_gk = pd.read_csv('/player_gk.csv')
fifa_teams = pd.read_csv('/fifa_teams.csv').apply(pd.to_numeric,errors='ignore')
fifa_teams.rename(columns={'Unnamed: 0':'Team'},inplace=True)
fifa_teams['Team'] = fifa_teams['Team'].str.replace('-',' ').replace('Korea Republic','South Korea').replace('United States','USA')


In [3]:
query = f'''

SELECT
  TeamId,
  MatchId,
  teams.team as team,
  PlayerId,
  players.player as Player,
  X,
  Y,
  Type,
  OutcomeType,
  SatisfiedEventsTypes,
  PassEndX,
  PassEndY,
  Zone,
  Length,
  Angle,
  Period,
  Minute,
  Second,
FROM
  `DATASET.Event_Data.World_Cup_2022` as wc
LEFT JOIN 
  `DATASET.Lookup_Tables.Players` as players ON wc.PlayerId = players.id
LEFT JOIN 
  `DATASET.Lookup_Tables.Teams` as teams ON wc.TeamId = teams.id
WHERE
isTouch = True


'''
all_touches_master = pd.read_gbq(query, project_id='DATASET')
query = f'''
WITH Touches as (
SELECT
  TeamId,
  MatchId,
  teams.team as team,
  PlayerId,
  players.player as Player,
  X,
  Y,
  Type,
  OutcomeType,
  SatisfiedEventsTypes,
  PassEndX,
  PassEndY,
  Zone,
  Length,
  Angle,
  Period,
  Minute,
  Second,
FROM
  `DATASET.Event_Data.World_Cup_2022` as wc
LEFT JOIN 
  `DATASET.Lookup_Tables.Players` as players ON wc.PlayerId = players.id
LEFT JOIN 
  `DATASET.Lookup_Tables.Teams` as teams ON wc.TeamId = teams.id
WHERE
isTouch = True)

SELECT * from Touches 
WHERE (
  SELECT count(1)
  FROM UNNEST(SatisfiedEventsTypes) as events
  WHERE events in (5,6,21,22,31,32,33,34,42,44,45,48,50,51,132,133,134,135,136,212)
) <= 0


'''
all_touches_open_play = pd.read_gbq(query, project_id='DATASET')

In [4]:
events_mapping = pd.read_gbq('SELECT * FROM `DATASET.Lookup_Tables.Event_Types`', project_id='DATASET')
events_mapping = events_mapping[['Id','Event']].set_index('Id').to_dict()['Event']

In [5]:
all_touches_master['SatisfiedEventsTypes'] = all_touches_master['SatisfiedEventsTypes'].apply(lambda x: [events_mapping[i] for i in x])

In [6]:
query = '''
SELECT
  TeamId,
  teams.team as team,
  PlayerId,
  players.player as Player,
  X,
  Y,
  Type,
  OutcomeType,
  event_types.Event as Event,
  Period,
  GoalMouthY,
  GoalMouthZ,
FROM
  `DATASET.Event_Data.World_Cup_2022` as wc,
  UNNEST(wc.SatisfiedEventsTypes) as Event_Id
LEFT JOIN 
  `DATASET.Lookup_Tables.Players` as players ON wc.PlayerId = players.id
LEFT JOIN 
  `DATASET.Lookup_Tables.Teams` as teams ON wc.TeamId = teams.id
LEFT JOIN
    `DATASET.Lookup_Tables.Event_Types` as event_types ON Event_Id = event_types.Id
WHERE
Event = 'shotOpenPlay'
'''
all_shots = pd.read_gbq(query, project_id='DATASET').drop('Event',axis=1).drop_duplicates()

In [7]:
teams = all_touches_open_play['team'].unique()
teams.sort()

In [ ]:
import matplotlib
matplotlib.get_cachedir()

In [136]:
#FORMATS COLORS 
TEAM_COLORS = {'Argentina':'Blue',
'Australia':'Yellow',
'Belgium':'Red',
'Brazil':'Yellow',
'Cameroon':'Green',
'Canada':'Red',
'Costa Rica':'Red',
'Croatia':'Red',
'Denmark':'Red',
'Ecuador':'Yellow',
'England':'Red',
'France':'Blue',
'Germany':'Red',
'Ghana':'Red',
'Iran':'Red',
'Japan':'Blue',
'Mexico':'Green',
'Morocco':'Red',
'Netherlands':'Orange',
'Poland':'Red',
'Portugal':'Red',
'Qatar':'Red',
'Saudi Arabia':'Green',
'Senegal':'Green',
'Serbia':'Red',
'South Korea':'Red',
'Spain':'Red',
'Switzerland':"Red",
'Tunisia':'Red',
'USA':'Blue',
'Uruguay':"Blue",
'Wales':'Red'
}

COLOR_MAPS = {
    'Green': ['#296600','#358600','#63C132'],
    'Orange': ['#7A2700','#CC4100','#FE621D'],
    'Blue': ['#0C7489','#22AAA1','#17D4DE'],
    'Red': ['#780214','#DD0426','#FB3640'],
    'Yellow': ['#F7B32B','#FFC100','#FFEAAE']
}

In [97]:
#Main formats
BG_COLOR = '#101419'
TITLE_FONT = 'Merriweather'
FONT = 'Source Sans Pro'
NOTES_COLOR = '#E0E1DD'
TITLE_COLOR = '#E0E1DD'
LINE ='#F9F5E3'
TITLE_SIZE = 28

### 1. Areas of control 

Idea 
1. Create a master dictionary where key is the team and the value is a np array
2. For each team, get a list of MatchIds involved
3. For each match id
    - Bin the touches 6 equal bins of X and 5 Juego Y then get the 
    - Count the number of touches in each x,y (similar to the xt method) for the team and all the opponents 
    - For i,j in zip(x_bins,y_bins):
        - set 1 if team -1 if opponent bigger than threshold else 0 if contested 



In [10]:
from matplotlib.patches import Rectangle, Polygon
import matplotlib.lines as lines

In [11]:
teams = all_touches_open_play['team'].unique()
teams.sort()
touch_dict = {}
x_bins = [0,17,33.5,50,66.5,83,100] #vertical bins 7 lines
y_bins = [0,21.1,36.8,63.2,78.9,100] #horizontal bins 6 lines
THESHOLD = 0.56
for team in teams:
    team_dict = {}
    matches = all_touches_open_play[all_touches_open_play['team'] == team]['MatchId'].unique()
    team_touch = all_touches_open_play[all_touches_open_play['team'] == team].copy()
    opponent = all_touches_open_play[(all_touches_open_play['MatchId'].isin(matches)) & (all_touches_open_play['team'] != team)].copy()
    opponent['X'] = (100 - opponent['X'])
    opponent['Y'] = (100 - opponent['Y'])
    team_touch['x_bin'] = pd.cut(team_touch['X'], bins=x_bins, labels=False)
    team_touch['y_bin'] = pd.cut(team_touch['Y'], bins=y_bins, labels=False)
    opponent['x_bin'] = pd.cut(opponent['X'], bins=x_bins, labels=False)
    opponent['y_bin'] = pd.cut(opponent['Y'], bins=y_bins, labels=False)
    array = np.empty((5,6),dtype='int')
    pctarray = np.empty((5,6),dtype='float')
    grouped = team_touch.groupby(['x_bin','y_bin']).size().to_frame().reset_index()
    grouped_opponent = opponent.groupby(['x_bin','y_bin']).size().to_frame().reset_index()
    for x_coord in range(0,len(x_bins) - 1): #6 
        for y_coord in range(0, len(y_bins) - 1): #5
            team_touches = int(grouped[grouped['x_bin'] == x_coord][grouped['y_bin'] == y_coord][0].values) 
            opponent_touches = int(grouped_opponent[grouped_opponent['x_bin'] == x_coord][grouped_opponent['y_bin'] == y_coord][0].values)
            total = team_touches + opponent_touches
            team_pct = team_touches/total
            opp_pct = opponent_touches/total
            if team_pct >= THESHOLD:
                array[y_coord,x_coord] = 1
                pctarray[y_coord,x_coord] = team_pct
            elif opp_pct >= THESHOLD:
                array[y_coord,x_coord] = -1
                pctarray[y_coord,x_coord] = team_pct
            else:
                array[y_coord,x_coord] = 0
                pctarray[y_coord,x_coord] = team_pct
            
        team_dict['touches'] = array
        team_dict['pct'] = pctarray
        
    touch_dict[team] = team_dict
    #cant use imshow because it will flip the y axis
    #this array is flipped because 0 ybin is the bottom of the pitch but its the first row of teh axis
    

In [12]:
def draw_one(ax,team,x_bins,y_bins,cmap):
    p = ax.pcolormesh(x_bins,y_bins,touch_dict[team]['touches'], cmap=cmap, vmin=-1, vmax=1,edgecolor =LINE,linewidth=0)
    ax.text(50,105,team.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    flag = ax.inset_axes([0.8,1.0,0.15,0.15])
    im = plt.imread(f'team}.png')
    flag.imshow(im)
    flag.axis('off')
    ax.vlines(x_bins,0,100,colors=LINE,linewidth=0.6,alpha=0.8)
    ax.hlines(y_bins,0,100,colors=LINE,linewidth=0.6,alpha=0.8)
    return p


In [ ]:
plt.rcParams['figure.figsize'] = [16,12]
NOTES_COLOR = '#E0E1DD'
TEXTBOX = 'grey' 
TITLE_COLOR = '#E0E1DD'
BAD_COLOR = '#FD3E81'
CONTESTED_COLOR = '#FE7F2D'
GOOD_COLOR = '#42BFDD'
cmap_lst = [BAD_COLOR, CONTESTED_COLOR, GOOD_COLOR]
TITLE_SIZE = 28
LINE ='#F9F5E3'


pitch = Pitch(pitch_type='opta',pitch_color = BG_COLOR,line_color=LINE,linewidth = 0.6, line_alpha=0.9,line_zorder=2,spot_scale=0.003)
fig,axs = plt.subplots(7,5)
fig.set_facecolor(BG_COLOR)
cmap = LinearSegmentedColormap.from_list('mycmap', cmap_lst)
for team, ax in zip(teams, axs.flatten()[:-3]):
    pitch.draw(ax=ax)
    draw_one(ax,team,x_bins,y_bins,cmap)
for ax in axs.flatten()[-3:]:
    ax.axis('off')

fig.text(0.125,1.02,' ')
fig.text(0.9,1.02,' ')
_ = fig.add_artist(lines.Line2D([0.14,0.883],[1.03],linewidth=1,color=TITLE_COLOR))
fig.text(0.14,0.985,'Zones of Control',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.14,0.965, '''Zones of Control highlights the percentage of open-play touches that a team has in a given zone of the pitch. <Team Zones> are areas where the team has 
more than 55% of the total touches in that zone while <Opponent Zones> are areas where the opponent has more than 55% of the total touches in that 
zone and the rest are denoted as <Contested Zones>.''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':GOOD_COLOR},{'color':BAD_COLOR},{'color':CONTESTED_COLOR}])

fig.text(0.885,0.155,'''Data via Opta: Opta defines touches as the number of times a player has touched 
the ball Receiving a pass, then dribbling,then sending a pass counts as one touch.

Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.885,0.12,'Inspired by @petermckeever and @johnspacemuller\nCreated by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

for x in [0.82,0.835,0.85]:
    t = Polygon([[x-0.007,0.96],[x,0.965],[x-0.007,0.97]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
    fig.add_artist(t)
fig.text(0.833,0.945,'Attacking Direction',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)
plt.savefig(f'Completed/WorldCupReview/Zones of Control.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

### 2. Shot Map 
idea 
1. Find the top 8 xg shotters (this one just hard code from fbref xg data)
2. Hex bin and median distance 

In [15]:
from matplotlib.patches import Arc


In [93]:
top8_players = fbref_players.sort_values(['Shooting Standard Sh'],ascending=False).head(8)['Standard Player'].tolist()
top8_teams = fbref_teams.sort_values(['Shooting Standard Sh/90'],ascending=False).head(8)['Standard Squad'].tolist()

In [72]:
def plot_one_shotmap(ax,df,team,cmap_lst):
    df = df[df['team'] == team]
    df = df[(df['Type'].isin(['MissedShots','ShotOnPost','SavedShot', 'Goal'])) & (df['Period'] != 6)].drop('SatisfiedEventsTypes',axis=1,errors='ignore').drop_duplicates()
    pitch = VerticalPitch(half=True,pitch_type='opta',pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,pad_bottom=0.5)
    pitch.draw(ax=ax)
    pitch.scatter(df['X'],df['Y'],ax=ax,color=df['Type'].apply(lambda x: cmap_lst[2] if x =='Goal' else 'gray'),s= 30 ,alpha = df['Type'].apply(lambda x: 1 if x =='Goal' else 0.4 ),linewidths=0,zorder =3)

    fontsize = 6.5
    fontweight = 'heavy'
    alpha = 0.75
    ax.text(90,70,'Goals:',bbox = dict(facecolor=cmap_lst[2],alpha=0.8,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha )
    ax.text(78,70,len(df[df['Type'] =='Goal']),fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha+0.25)
    ax.text(90,66,'Shots:',bbox = dict(facecolor='grey',alpha=alpha-0.2,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(78,66,len(df),fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(90,62,'xG/Shot:',bbox = dict(facecolor='grey',alpha=alpha-0.2,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(77,62,fbref_teams[fbref_teams['Standard Squad'] == team]['Shooting Expected Npxg/Sh'].values[0],fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)

    df['Dist'] = (df['X'] - 100)**2 + (df['Y'] - 50)**2
    df['Dist'] = df['Dist'].apply(lambda x: np.sqrt(x))
    a = Arc((50,100),df['Dist'].median() * 2 * 105/100,df['Dist'].median() * 2 * 68/100,theta1 = 180, theta2=0,color=None,edgecolor=RANGE_COLOR,zorder=3,linewidth=1,linestyle='dotted')
    ax.add_patch(a)
    m = df['Dist'].median()
    ax.text(0,48, f'Median Dist: {round(m * 1.25,1)}m',fontsize=8,ha='right',va='top',color = RANGE_COLOR,font=FONT,alpha = 0.7)
    ax.text(50,103.5,team.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    logoax = ax.inset_axes([0.8,0.98,0.12,0.12])
    im = plt.imread(f'{team}.png')
    logoax.imshow(im)
    logoax.axis('off')
    return ax


In [ ]:
RANGE_COLOR ='#DDAE7E'

fig,axs  = plt.subplots(2,4,figsize=(20,8))
fig.set_facecolor(BG_COLOR)
for ax,team in zip(axs.flatten(),top8_teams):
    ax = plot_one_shotmap(ax,all_shots,team,COLOR_MAPS.get(TEAM_COLORS.get(team)))

_ = fig.add_artist(lines.Line2D([0.12,0.9],[1.05],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.985,'Shooting Locations',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.95, '''Shotmaps of the 8 teams with the highest number of shots per 90 recorded in the tournament and their <median> shot distance from goal''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':RANGE_COLOR}])

fig.text(0.9,0.08,'''Data via Opta. Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.06,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)
plt.savefig(f'Completed/WorldCupReview/TeamShotMaps.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

### 3. Goal Kick Sonars 
KDE Plot of End Locations?

In [100]:
keeper_team_mapped = pd.Series(fbref_gk['Goalkeeping Squad'].values, index=fbref_gk['Goalkeeping Player']).to_dict()
keepers = all_touches_open_play[all_touches_open_play['Player'].isin(keeper_team_mapped.keys())]

In [101]:
from matplotlib.transforms import Affine2D

In [102]:
def draw_keepermap_control(ax, df,team):
    keepers_name = fbref_gk[fbref_gk['Goalkeeping Squad'] == team]['Goalkeeping Player'].unique()
    keepers = df[df['Player'].isin(keepers_name)].copy()
    pitch = Pitch(pitch_type='opta',pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,line_zorder=2,pad_bottom=-0.5)
    pitch.draw(ax = ax)
    # pitch.kdeplot(keepers['PassEndX'],keepers['PassEndY'],ax=ax,fill = True, levels = 200,cmap=cmap))
    def get_player_df(player_df,bins):
        angle = player_df['Angle']
        player_df['Bins'] = pd.DataFrame(pd.cut(angle,bins=bins,include_lowest=True))
        length = player_df.groupby('Bins').mean(numeric_only=True)['Length']
        counts = player_df['Bins'].value_counts()
        working_df = pd.concat([length,counts],axis=1).reset_index()
        working_df['Midpoint'] = working_df['index'].apply(lambda x:x.mid)
        working_df.drop('index',axis=1,inplace=True)
        working_df.columns = ['Mean','Count','Midpoint']
        working_df['Scaled Length'] = working_df['Mean']
        mean_x = player_df['X'].mean()
        mean_y = player_df['Y'].mean()
        return player_df, working_df.fillna(0), mean_x,mean_y
    player_df, working_df,x,y = get_player_df(keepers,20)
    working_df= working_df[(working_df['Midpoint'].astype(float) < np.deg2rad(90)) | (working_df['Midpoint'].astype(float) > np.deg2rad(270))]
    working_df['deg'] = working_df['Midpoint'].apply(lambda x:np.rad2deg(x))
    print(working_df)
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    colors = plt.cm.ScalarMappable(cmap='Reds').to_rgba(working_df['Count']/working_df['Count'].mean())
    newax = ax.inset_axes([-1,-1,3,3],projection='polar')
    newax.set_theta_zero_location("E")
    newax.set_thetamax(90)
    newax.set_theta_direction('clockwise')
    newax.set_thetamin(-90)    
    newax.set_ylim(0,100)
    newax.bar(working_df['Midpoint'],working_df['Scaled Length'],color = colors,width=0.2)
    newax.axis('off')
    
    return b


In [148]:
def draw_keepermap(ax, df,team,bins,width,cmap_lst,kde_cmap_lst):
    keepers_name = fbref_gk[fbref_gk['Goalkeeping Squad'] == team]['Goalkeeping Player'].unique()
    keepers = df[df['Player'].isin(keepers_name)].copy()
    pitch = Pitch(pitch_type='opta',line_zorder=2,pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,pad_top=10)
    pitch.draw(ax = ax)
    # pitch.kdeplot(keepers['PassEndX'],keepers['PassEndY'],ax=ax,fill = True, levels = 200,cmap=cmap))
    kde_cmap = LinearSegmentedColormap.from_list('test',kde_cmap_lst, N=256)
    def get_player_df(player_df,bins):
        angle = player_df['Angle']
        player_df['Bins'] = pd.DataFrame(pd.cut(angle,bins=bins,include_lowest=True))
        length = player_df.groupby('Bins').mean(numeric_only = True)['Length']
        counts = player_df['Bins'].value_counts()
        working_df = pd.concat([length,counts],axis=1).reset_index().fillna(0)
        working_df['Midpoint'] = working_df['index'].apply(lambda x:x.mid)
        working_df.drop('index',axis=1,inplace=True)
        working_df.columns = ['Mean','Count','Midpoint']
        working_df['Scaled Length'] = working_df['Mean']#**2 * 5
        mean_x = player_df['X'].mean()
        mean_y = player_df['Y'].mean()
        return player_df, working_df.fillna(0), mean_x,mean_y
    player_df, working_df,x,y = get_player_df(keepers,bins)
    working_df= working_df[(working_df['Midpoint'].astype(float) < np.deg2rad(85)) | (working_df['Midpoint'].astype(float) > np.deg2rad(280))]
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba(working_df['Count']/working_df['Count'].mean())

    working_df['deg'] = working_df['Midpoint'].apply(lambda x:np.rad2deg(x))
    #pitch.kdeplot(keepers['PassEndX'],keepers['PassEndY'],ax=ax,fill = True, levels = 200,cmap=kde_cmap,alpha=0.6)
    # pitch.scatter(keepers['PassEndX'],keepers['PassEndY'],ax=ax, s=5, c='white',alpha=0.5)
    for i in range(len(working_df)):
        length = working_df['Scaled Length'].iloc[i]
        midpoint = working_df['Midpoint'].iloc[i]
        coords = [(length, 50 + width/2),(0,50),(length,50 - width/2)]
        original = Polygon(coords,facecolor = colors[i],edgecolor = None,alpha=1,zorder = 1)
        transform = Affine2D().rotate_deg_around(0,50,-np.rad2deg(midpoint)) + ax.transData
        original.set_transform(transform)
        ax.add_patch(original)
    flag = ax.inset_axes([0.8,0.96,0.15,0.15])
    im = plt.imread(f'{team}.png')
    flag.imshow(im)
    flag.axis('off')
    ax.text(50,105,team.upper(),fontsize=13,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    
    ax.vlines(keepers['X'].mean(),0,100,linestyle='--',color='white',linewidth=0.5)
    m = round(keepers['X'].mean(),1)
    ax.text(0,-7,f'Mean touch distance: {m}',fontsize=7,ha='left',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    
    return ax


In [ ]:

cmap_lst = ['#06BA63','#0FFF95','#8CFBDE']

fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)

for ax,team in zip(axs.flatten()[:-3],teams):
    ax = draw_keepermap(ax,all_touches_open_play,team,30,6,cmap_lst,COLOR_MAPS.get(TEAM_COLORS.get(team)))
for ax in axs.flatten()[-3:]:
    ax.axis('off')

fig.text(0.11,0.97,' ')
fig.text(0.92,0.97,' ')
_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.965],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.945,'Goalkeepers in Buildup',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.935, '''Goalkeepers are now more involved in buildup play than ever before. This visual highlights the types of passes 
goalkeepers are making in buildup play and the ending locations of these passes. Length of each bar represents 
the average length of the pass, with <lighter> colors indicating a higher frequency of passes while <darker> colors
indicate fewer passes were made of that type. ''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':cmap_lst[-1]},{'color':cmap_lst[0]}])

fig.text(0.9,0.135,'''Data via Opta. Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.125,'Inspired by @jonollington | Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

for x in [0.82,0.835,0.85]:
    t = Polygon([[x-0.007,0.925],[x,0.93],[x-0.007,0.935]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
    fig.add_artist(t)
fig.text(0.833,0.91,'Attacking Direction',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)
plt.savefig(f'Completed/WorldCupReview/GKSonar.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

### 4. Scatter Plot Ideas 
- Dribble attempted vs won %
- Shot Prevented vs xg 
- Goals Prevented / xGot (bar)
- PPDA vs High Turnovers (?)


In [ ]:
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes

In [ ]:
def create_tilt_ax():
    def setup_axes1(fig, rect,edge_color):
        tr = Affine2D().scale(2, 2).rotate_deg(45)

        grid_helper = floating_axes.GridHelperCurveLinear(
            tr, extremes=(0, 101, 0, 101))

        ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
        fig.add_subplot(ax1)

        aux_ax = ax1.get_aux_axes(tr)
        for key in ax1.axis:
            # ax1.axis[key].set_visible(False)
            ax1.axis[key].major_ticklabels.set_visible(False)
            ax1.axis[key].toggle(all=False,label=True)
            ax1.axis[key].line.set_color(edge_color)
        return ax1, aux_ax
    fig,ax = plt.subplots()
    fig.set_size_inches(10,10)
    ax.set_aspect('equal', 'box')
    ax.axis('off')
    fig.patch.set_facecolor(BG_COLOR)
    ax1,tilt_ax = setup_axes1(fig, 111,'black')
    ax1.set_facecolor(BG_COLOR)
    ax1.get_xaxis().set_visible(False)
    plt.grid(color='grey',alpha=0.5, linestyle='-', linewidth=1)
    tilt_ax.plot([50,50,101],[101,50,50],color = 'black',linewidth = 1)
    rect = Rectangle((50,50),50,50,facecolor=GOOD_COLOR,alpha=0.1)
    tilt_ax.add_patch(rect)
    return fig,ax,tilt_ax
fig,ax,tilt_ax = create_tilt_ax()
tilt_ax.scatter(the_analyst_players['Take-Ons Attempts'],the_analyst_players['Take-Ons Win %'],color='white',s=100,edgecolor='black',linewidth=0.5)

### 5.Defence 
pitch.bin_statistic(team_touch['X'],team_touch['Y'], statistic='count', bins=(25,25))['statistic'].shape
- location and size of the heatmap scatter represents the location and frequency of the defensive actions 
- color represnts the success rate


In [146]:
from scipy.ndimage import gaussian_filter

In [147]:
pd.options.mode.chained_assignment = None  # default='warn'

In [150]:
def plot_one_defence_control(df,team,ax):
    defence = df[(df['team'] == team) & (df['Type'].isin(['Interception','Clearance','BlockedPass','Disposessed','Tackle']))].drop('SatisfiedEventsTypes',axis=1).drop_duplicates().copy()
    pitch = Pitch(pitch_type='opta',pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,line_zorder=2,pad_bottom=-0.5)
    pitch.draw(ax = ax)
    # bin_statistic = pitch.bin_statistic(defence['X'], defence['Y'],statistic='count', bins=(25, 25))
    # bin_statistic['statistic'] = gaussian_filter(bin_statistic['statistic'], 1)
    # pcm = pitch.heatmap(bin_statistic, ax=ax, cmap='hot', edgecolors='#22312b')
    pitch.scatter(defence['X'],defence['Y'],c='red',s=10,ax=ax)


In [151]:
def process_change(df):
    raw = df.drop('SatisfiedEventsTypes',axis=1).drop_duplicates().sort_values(['MatchId','Period','Minute','Second']).reset_index(drop=True).copy()    
    change = [False,False]
    raw = raw[raw['Type'].isin(['Interception','BlockedPass','Disposessed','Tackle','Pass'])].reset_index(drop=True)
    for i,row in raw.iloc[2:].iterrows():
        prev = raw.loc[i-1]
        prev_s = raw.iloc[i-2]
        if ((prev['Type'] in['Tackle','Interception','BlockedPass','Disposessed']) and (prev['OutcomeType'] == True) and(prev['team'] == row['team'])  or ( (prev_s['Type'] in['Tackle','Interception','BlockedPass','Disposessed']) and (prev_s['OutcomeType'] == True)) and prev_s['team'] == row['team']) and (row['Type']=='Pass'):
            change.append(True)
        else:
            change.append(False)
    raw['Change'] = change
    return raw
change_posession = process_change(all_touches_open_play)

In [166]:
def plot_defence(df,main,team,ax,cmap_lst):
    pitch = Pitch(pitch_type='opta',line_zorder=2,pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,pad_top=10)
    pitch.draw(ax=ax)
    flag = ax.inset_axes([0.8,0.95,0.15,0.15])
    im = plt.imread(f'{team}.png')
    flag.imshow(im)
    flag.axis('off')
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    defence = main[(main['team'] == team) & (main['Type'].isin(['Interception','BlockedPass','Disposessed','Tackle','Clearance']))].drop('SatisfiedEventsTypes',axis=1).drop_duplicates().copy()
    pitch.kdeplot(defence['X'],defence['Y'],cmap=cmap,ax=ax,fill=True,shade_lowest=False,alpha=0.8,levels = 500)
    #pitch.hexbin(defence['X'],defence['Y'],cmap=cmap,ax=ax,gridsize=(12,8),alpha=0.8)
    mean_x = defence['X'].mean() * 1.25
    ax.text(100,-6,f'Average Height of Def. Actions: {mean_x:.2f}m',ha='right',va='center',color=NOTES_COLOR,alpha=0.75,fontsize = 6.75,font = FONT)
    # ax.vlines(mean_x,-8,105,NOTES_COLOR,linewidth=1,linestyle='dotted')
    smallx = defence['X'].quantile(0.35)
    largex = defence['X'].quantile(0.65)
    # rect = Rectangle((smallx,0),largex-smallx,100,facecolor = RECT_COLOR, alpha=0.7,edgecolor='none',zorder=2)
    # ax.add_patch(rect)
    ax.vlines([smallx,largex],-0,100,RECT_COLOR,linewidth=1,linestyle='--')
    ax.text(50,105,team.upper(),fontsize=13,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    return ax


In [ ]:
fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)
cmap_lst = [BG_COLOR,'#B01C5A','#DF367C','#FF4D80']
RECT_COLOR = '#3CBBB1'

for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_defence(change_posession,all_touches_open_play,team,ax,cmap_lst)

for ax in axs.flatten()[-3:]:
    ax.axis('off')

fig.text(0.11,0.97,' ')
fig.text(0.92,0.97,' ')
_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.965],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.94,'Defensive Zones',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.93, '''This visual highlights the areas of the pitch where teams <defend> where the areas between <dashed lines> represents 
the locations where the middle 30% of the defensive actions occur. Defensive actions considered here are successful 
intereceptions, blocked passes, tackles and clearances. ''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':cmap_lst[2]},{'color':RECT_COLOR}])

fig.text(0.9,0.140,'''Data via Opta. 

Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.125,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

for x in [0.82,0.835,0.85]:
    t = Polygon([[x-0.007,0.925],[x,0.93],[x-0.007,0.935]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
    fig.add_artist(t)
fig.text(0.833,0.91,'Attacking Direction',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)
plt.savefig(f'Completed/WorldCupReview/DefensiveZones.jpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

In [165]:
def plot_transition(df,cmap_lst,team,ax):
    def sonar_plot(newax,df,cmap_lst):
        def get_player_df(player_df,bins):
            angle = player_df['Angle']
            player_df['Bins'] = pd.DataFrame(pd.cut(angle,bins=bins,include_lowest=True))
            length = player_df.groupby('Bins').mean(numeric_only=True)['Length']
            counts = player_df['Bins'].value_counts()
            working_df = pd.concat([length,counts],axis=1).reset_index()
            working_df['Midpoint'] = working_df['index'].apply(lambda x:x.mid)
            working_df.drop('index',axis=1,inplace=True)
            working_df.columns = ['Mean','Count','Midpoint']
            working_df['Scaled Length'] = working_df['Mean']
            mean_x = player_df['X'].mean()
            mean_y = player_df['Y'].mean()
            return player_df, working_df.fillna(0), mean_x,mean_y
        player_df, working_df,x,y = get_player_df(df,18)
        cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
        colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba(working_df['Count']/working_df['Count'].mean())
        newax = ax.inset_axes([0.07,0.07,0.86,0.86],projection='polar')
        newax.set_theta_zero_location("N")
        newax.bar(working_df['Midpoint'],working_df['Scaled Length'],color = colors,width=0.35)
        newax.set_facecolor(BG_COLOR)
        newax.set_rticks([])
        newax.set_yticklabels([])
        newax.set_xticklabels([])
        newax.grid(False)
        newax.spines['polar'].set_color(NOTES_COLOR)
        newax.set_ylim(0,50)
        return
    df = df[df['team'] == team]
    changed = df[df['Change'] == True]
    sonar_plot(ax,changed,sonar_cmap)
    ax.text(0.5,1,team.upper(),fontsize=10,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    ax.axis('off')
    return ax


In [173]:
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

In [186]:
fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)
cmap_lst = [BAD_COLOR,CONTESTED_COLOR,GOOD_COLOR]
RECT_COLOR = 'blue'

for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_transition(change_posession,cmap_lst,team,ax)

for ax in axs.flatten()[-3:]:
    ax.axis('off')

fig.text(0.11,0.97,' ')
fig.text(0.92,0.97,' ')
_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.965],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.94,'Transitional Play',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.93, '''This visual highlights how teams transition from defence to attack. The pass sonars indicate the type of transition passes
each team prefers. Transition passes are the first two passes made upon winning back posession after a successful defensive 
action. <Darker> bars indicate higher frequency of passes, while <lighter> bars indicate fewer passes of that type.''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':cmap_lst[0]},{'color':cmap_lst[-1]}])

fig.text(0.9,0.155,'''Data via Opta. Defensive actions considered here are successful intereceptions, blocked passes, 
tackles and disposesssion. Clearances are not considered here as they often do not represent 
a conscious effort to send the ball to a particular area of the pitch. The radius of each plot
is also standardized to allow for distance comparisons between teams

Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.125,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

# for x in [0.82,0.835,0.85]:
#     t = Polygon([[x-0.007,0.925],[x,0.93],[x-0.007,0.935]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
#     fig.add_artist(t)
t = Polygon([[0.823,0.92],[0.833,0.93],[0.843,0.92]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
fig.add_artist(t)
fig.text(0.833,0.91,'Attacking Direction',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)


cbar_ax = axs[6,2].inset_axes([0,-0.3,1,1])
cbar_ax.set_facecolor(BG_COLOR)
cmappable = ScalarMappable(norm=Normalize(0,1), cmap=cmap)
cbar = plt.colorbar(cmappable,orientation = 'horizontal',ax=cbar_ax,fraction=0.9,location='bottom')
cbar_ax.axis('off')
cbar_ax.tick_params(size=0)
cbar.set_ticks([])
cbar.outline.set_visible(False)
fig.text(0.45,0.15,'Lower Frequency',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)
fig.text(0.57,0.15,'Higher Frequency',font=FONT,color = NOTES_COLOR,fontsize = 8,fontweight ='roman',ha='center',va='center',alpha =0.8)
plt.savefig(f'Completed/WorldCupReview/Transitionjpg',bbox_inches='tight',facecolor=fig.get_facecolor(), edgecolor='none', dpi=300)

### 6. xT Plots

In [ ]:
from matplotlib.patches import Circle

In [ ]:
xt_grid = np.array(pd.read_json('https://karun.in/blog/data/open_xt_12x8_v1.json'))
xT_rows, xT_cols = xt_grid.shape

def get_xt_players(df):
    def process_df(df):
        df = df.sort_values(['MatchId', 'Period','Minute','Second']).drop(['Events','SatisfiedEventsTypes','Zone'],errors='ignore',axis=1).drop_duplicates().reset_index(drop=True)
        df['TotalTime'] = df['Minute']*60 + df['Second']
        passed_from = [None]
        for i,row in df.iloc[1:].iterrows():
            prev = df.loc[i - 1]
            if (prev['Type'] == 'Pass') & (prev['OutcomeType'] == True) & (prev['MatchId'] == row['MatchId']) & (abs(prev['TotalTime'] - row['TotalTime']) < 10):
                passed_from.append(prev['Player'])
            else:
                passed_from.append(None)

        passed_to  = []
        for i,row in df.iterrows():
            if (row['Type'] == 'Pass') & (row['OutcomeType'] == True) & (row['MatchId'] == row['MatchId']) & (abs(row['TotalTime'] - row['TotalTime']) < 10):
                passed_to.append(df.loc[i+1]['Player'])
            else: 
                passed_to.append(None)
        df['PassedFrom'] = passed_from
        df['PassedTo'] = passed_to
        return df
    
    def calc_xt(df,x_start,x_end,y_start,y_end,name):
        df['x1_bin'] = pd.cut(df[x_start], bins=xT_cols, labels=False)
        df['y1_bin'] = pd.cut(df[y_start], bins=xT_rows, labels=False)
        df['x2_bin'] = pd.cut(df[x_end], bins=xT_cols, labels=False)
        df['y2_bin'] = pd.cut(df[y_end], bins=xT_rows, labels=False)
        df['start_zone_value'] = df[['x1_bin', 'y1_bin']].apply(lambda x: xt_grid[x[1]][x[0]], axis=1)
        df['end_zone_value'] = df[['x2_bin', 'y2_bin']].apply(lambda x: xt_grid[x[1]][x[0]], axis=1)
        df[name] = df['end_zone_value'] - df['start_zone_value']
        df = df.drop(['x1_bin', 'y1_bin','x2_bin', 'y2_bin','start_zone_value','end_zone_value'],axis=1)
        return df[['Player',name]].groupby('Player').sum().sort_values(name,ascending=False).reset_index(drop=False)

    #xt by pass
    processed = process_df(df)
    df_passes = processed[(~processed['PassEndX'].isna()) & (processed['OutcomeType'] == True)].copy()
    xt_passes = calc_xt(df_passes,'X','PassEndX','Y','PassEndY','xT_pass')
    #xt by carries
    fromx = [None]
    fromy =[None]
    for i,row in processed.iloc[1:].iterrows():
        prev = processed.loc[i - 1]
        if (row['PassedFrom'] != None) & (prev['MatchId'] == row['MatchId']) & (prev['Period'] == row['Period']):
            fromx.append(prev['PassEndX'])
            fromy.append(prev['PassEndY'])
        else:
            fromx.append(None)
            fromy.append(None)
    processed['FromX'] = fromx
    processed['FromY'] = fromy
    processed = processed[processed['FromX'].notna()]
    xt_carries = calc_xt(processed,'FromX','X','FromY','Y','xT_carry')
    xt = pd.merge(xt_passes,xt_carries,on='Player',how='outer')
    xt['xT'] = xt['xT_pass'] + xt['xT_carry']
    xt = xt.sort_values('xT',ascending=False)
    return xt
xt = get_xt_players(all_touches_master)
player_team_mapped = pd.Series(all_touches_master['team'].values, index=all_touches_master['Player']).to_dict()
xt['Team'] = xt['Player'].map(player_team_mapped)
teams = all_touches_master['team'].unique()

In [ ]:
def plot_xt_chart(xt,ax,team,cmap_lst):
    xt = xt[(xt['xT'] > 0) &(xt['Team'] == team)].copy().reset_index(drop=True)
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256)
    colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba(xt['xT']/xt['xT'].mean())
    xt['Pct'] = xt['xT']/xt['xT'].sum() * 100
    #XT_THRESHOLD = xt['Pct'].sort_values(ascending=False).iloc[NUM_PLAYERS_TO_SHOW]
    XT_THRESHOLD = 5
    xt_label = xt.apply(lambda x: x['Player'] + ' (' + str(round(x['xT'],2))  + ')' if x['Pct'] > XT_THRESHOLD else '',axis=1)
    wedges,texts, labels = ax.pie(xt['xT'],colors=colors,autopct=lambda pct: '{:1.1f}%'.format(pct)  if pct > XT_THRESHOLD   else '' ,startangle=90,pctdistance =0.75,explode = [0.1]*len(xt)
                                                                                        ,textprops={'color':NOTES_COLOR,'fontweight':'demibold','font':FONT,'fontsize':4})


    for color,text,label in zip(colors,texts,labels):
        label.set_fontsize(7)
        label.set_fontweight('demi')
        if ((lambda c: 0.2126*c[0] + 0.7152*c[1] + 0.0722*c[2])(color) > 0.75 ):
            label.set_color('black')
            label.set_alpha(0.35)
    kw = dict(arrowprops=dict(arrowstyle="-",color=NOTES_COLOR,lw=0.4), zorder=0, va="center")
       
    NUM_PLAYERS_TO_SHOW = len(xt[xt['Pct'] > XT_THRESHOLD])
    for i, p in enumerate(wedges[:NUM_PLAYERS_TO_SHOW]):
        if labels[i].get_text() != '':
            ang = (p.theta2 - p.theta1)/2. + p.theta1 - 0.05
            y = np.sin(np.deg2rad(ang))
            x = np.cos(np.deg2rad(ang))
            horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
            connectionstyle = "angle,angleA=0,angleB={}".format(ang)
            kw["arrowprops"].update({"connectionstyle": connectionstyle})
            textxy = (1.35*np.sign(x), 1.35*y)
            if ('Mikkel Damsgaard' in xt_label[i]) | ('Vinícius' in xt_label[i]) | ('Msakni' in xt_label[i]) | ('Rafael Leão' in xt_label[i]) :
                textxy = (textxy[0]-0.2,textxy[1] -0.2)
            t = ax.annotate(xt_label[i], xy=(x, y), xytext=textxy, wrap = True,
                        horizontalalignment=horizontalalignment,color = NOTES_COLOR, fontsize = 5,**kw)
            t._get_wrap_line_width = lambda : 60
    centre_circle = Circle((0,0),radius =0.6,facecolor=BG_COLOR,edgecolor=BG_COLOR)
    ax.add_patch(centre_circle)
    ax.text(0,1.38,team.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    return ax 


In [ ]:
NUM_PLAYERS_TO_SHOW = 5
cmap_lst = [BG_COLOR,'#0C7489','#22AAA1','#17D4DE']


fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)
for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_xt_chart(xt,ax,team,COLOR_MAPS.get(TEAM_COLORS.get(team)))
for ax in axs.flatten()[-3:]:
    ax.axis('off')

plt.subplots_adjust(hspace=0.4)

fig.text(0.11,0.09,' ')
fig.text(0.92,0.985,' ')
_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.975],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.95,'Creative Outlets',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig.text(0.125,0.92, '''Expected Threat (xT) models quantify the change in probabilty of a team scoring as they move the ball to 
different areas of the pitch. This visual highlights who the creative outlets are for each team - the players that are creating 
the most xT for their teams''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman')

fig.text(0.9,0.13,'''Data via Opta, xT model by @karun1710. It is worth noting that xT is heavily influenced by set pieces and minutes 
played. However, given that this visual aims to highlight creative reliance of each team, players with 
higher minutes and set piece involvement are included as these metrics also indicate reliance.

Data is up to the World Cup Semifinals''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.11,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)


### 7. Crosses

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
intoPen = all_touches_open_play[(all_touches_open_play['Type'] == 'Pass') & (all_touches_open_play['OutcomeType'] == True) & (all_touches_open_play['PassEndX'] >= 83) & (all_touches_open_play['PassEndY'] <= 79) & (all_touches_open_play['PassEndY'] >= 21) & ((all_touches_open_play['X'] < 83) | (all_touches_open_play['Y'] > 79) | (all_touches_open_play['Y'] < 21))].drop('SatisfiedEventsTypes',axis=1).drop_duplicates()

In [ ]:
def plot_team(ax,team_name,pitchcolor,linecolor,leftcolor,rightcolor,edgecolor,pointcolor):
    pitch = VerticalPitch(half=True,pitch_type='opta',pad_bottom=-20,pad_top=15,pitch_color=pitchcolor,line_color=linecolor,linewidth=0.6,line_alpha=0.8)
    pitch.draw(ax=ax)
    def plot_arrows(X,ax,arrowcolor):
        kmeans = KMeans(n_clusters=2)
        kmeans.fit(X)
        X['cluster'] = kmeans.labels_
        largest = X['cluster'].value_counts().index[0]
        tmp = X[X['cluster']==largest]
        # pitch.arrows(tmp['X'],tmp['Y'],tmp['PassEndX'],tmp['PassEndY'],ax=ax,width=1.2,color=arrowcolor)
        pitch.lines(tmp['X'],tmp['Y'],tmp['PassEndX'],tmp['PassEndY'],ax=ax,lw=0.8,color=arrowcolor)
        pitch.scatter(tmp['PassEndX'],tmp['PassEndY'],ax=ax,color=arrowcolor,s=20,linewidths=0)
        tmp = X[X['cluster']!=largest]
        # pitch.arrows(tmp['X'],tmp['Y'],tmp['PassEndX'],tmp['PassEndY'],ax=ax,alpha=0.1,zorder=-1,width=1,color='#D1BE9C')
        pitch.lines(tmp['X'],tmp['Y'],tmp['PassEndX'],tmp['PassEndY'],ax=ax,alpha=0.2,zorder=-1,lw=0.6,color='#D1BE9C')
        pitch.scatter(tmp['PassEndX'],tmp['PassEndY'],ax=ax,color='#D1BE9C',s=10,alpha=0.2,zorder=-1,linewidths=0)
        return ax 

    team = intoPen[intoPen['team']==team_name]
    x1 = team[team['Y']>50]['Y'].mean()
    x2 = team[team['Y']<50]['Y'].mean()
    y1 = team[team['Y']>50]['X'].mean()
    y2 = team[team['Y']<50]['X'].mean()
    ax.scatter((x1,x2),(y1,y2),s=30,zorder=5,c=pointcolor)
    ax.plot((x1,x2),(y1,y2),zorder=4,linewidth=0.8,c=pointcolor,linestyle = '--')
    

    #text
    # dist = round(math.dist((x1,y1),(50,100)),1)
    ax.text(x1,y1-4, round(100-(y1* 1.25),1),ha='center',va='center',color=NOTES_COLOR,fontsize=8,font=FONT,fontweight='roman',bbox=dict(facecolor=pitchcolor,alpha=0.2,edgecolor='none',boxstyle='round4'))
    # dist = round(math.dist((x2,y2),(50,100)),1)
    ax.text(x2,y2-4,round(100-(y2* 1.25),1),ha='center',va='center',color=NOTES_COLOR,fontsize=8,font=FONT,fontweight='roman',bbox=dict(facecolor=pitchcolor,alpha=0.2,edgecolor='none',boxstyle='round4'))
    #fit both sides 
    X = team[team['Y'] > 50 ][['X','Y','PassEndX','PassEndY']]
    ax = plot_arrows(X,ax,rightcolor)
    X = team[team['Y'] < 50 ][['X','Y','PassEndX','PassEndY']]
    ax = plot_arrows(X,ax,leftcolor)

    #WIDTH IS JUST X DIFFERENTIAL; HEIGHT IS 100 - y (IE DISTANCE FROM GOAL)
    width = round(abs(x1-x2),1)
    ax.text(0,60,f'Attacking width: {width}m',ha='right',va='center',color=NOTES_COLOR,fontsize=7,alpha=0.7,font=FONT,fontweight='roman')

    ax.text(70,102,team_name.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    logoax = ax.inset_axes([0.8,0.82,0.12,0.12])
    im = plt.imread(f'{team_name}.png')
    logoax.imshow(im)
    logoax.axis('off')
    return ax

In [ ]:
fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)

POINT_COLOR = CONTESTED_COLOR
RIGHT_COLOR = BAD_COLOR
LEFT_COLOR = GOOD_COLOR
for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_team(ax,team,BG_COLOR,NOTES_COLOR,LEFT_COLOR,RIGHT_COLOR,NOTES_COLOR,POINT_COLOR)
for ax in axs.flatten()[-3:]:
    ax.axis('off')
plt.subplots_adjust(hspace=-0.15)

_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.918],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.895,'Crosses into the Penalty Box',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.885, '''This visual highlights the type of crosses into the penalty box for each team. The arrows represent the most common type of cross
from the <left> and <right> flank respectively. The <points> represent the average starting distance of crosses from each flank.''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':LEFT_COLOR},{'color':RIGHT_COLOR},{'color':POINT_COLOR}])

fig.text(0.9,0.14,'''Data via Opta. The K-Means clustering algorithm was used to identify the most common cross patterns, setting the number of clusters
to 2 from each flank. However, for teams register very few crosses, two clusters would lead to results which seem a little off.''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.13,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

### 8.Final Third Receipts
-Using the Fifapro data


In [ ]:
def plot_one_teams_reception(ax, df,team,cmap_lst):
    team_df = df[df['Team']==team]
    central = team_df['Final Third Entries Reception Central Channel'].values[0]
    inside_left = team_df['Final Third Entries Reception Inside Left Channel'].values[0]
    inside_right = team_df['Final Third Entries Reception Inside Right Channel'].values[0]
    left = team_df['Final Third Entries Reception Left Channel'].values[0]
    right = team_df['Final Third Entries Reception Right Channel'].values[0]
    total = central + inside_left + inside_right + left + right
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba(np.array([right,inside_right,central,inside_left,left])/total * 100)
    
    pitch = VerticalPitch(half=True,pitch_type='opta',pad_bottom=-20,pitch_color=BG_COLOR,line_color=NOTES_COLOR,linewidth=0.5,line_alpha=0.6)
    pitch.draw(ax=ax)
    x_coord = [10.55, 28.95, 50.  , 71.05, 89.45]
    ax.vlines([0,21.1,36.8,63.2,78.9,100],50,100,color=NOTES_COLOR,linewidth=0.6,alpha=0.6,linestyles='dotted')
    bar_ax = ax.inset_axes([0.037,0,0.926,0.8])
    bar_ax.bar(x_coord,height=[right,inside_right,central,inside_left,left],bottom=50,width=10,color=colors)
    for i, v in enumerate([right,inside_right,central,inside_left,left]):
        bar_ax.text(x_coord[i], 50 + v + 2, str(v), color=NOTES_COLOR, fontweight='heavy',ha='center',va='bottom',font=FONT,fontsize=8)
    
    ax.text(70,102,team.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    logoax = ax.inset_axes([0.8,0.98,0.12,0.12])
    im = plt.imread(f'{team}.png')
    logoax.imshow(im)
    logoax.axis('off')
    bar_ax.axis('off')
    return ax



In [ ]:
fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)
for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_one_teams_reception(ax,fifa_teams,team,COLOR_MAPS.get(TEAM_COLORS.get(team)))
for ax in axs.flatten()[-3:]:
    ax.axis('off')


_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.92],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.895,'Final Third Entries',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig.text(0.126,0.88,'Data via Fifa, up to Semifinals  |  Created by @nrehiew',ha='left',font = FONT,fontsize = 9, color = NOTES_COLOR,alpha=1)

# fig.text(0.89,0.15,'@nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)
# fig.text(0.89,0.16,'Data via Fifa, up to Semifinals',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)
plt.subplots_adjust(hspace=-0.15)

### 9. Penalty Shot Goalmouth

In [ ]:
query = '''
SELECT
  TeamId,
  teams.team as team,
  PlayerId,
  players.player as Player,
  X,
  Y,
  Type,
  OutcomeType,
  event_types.Event as Event,
  Period,
  GoalMouthY,
  GoalMouthZ,
FROM
  `DATASET.Event_Data.World_Cup_2022` as wc,
  UNNEST(wc.SatisfiedEventsTypes) as Event_Id
LEFT JOIN 
  `DATASET.Lookup_Tables.Players` as players ON wc.PlayerId = players.id
LEFT JOIN 
  `DATASET.Lookup_Tables.Teams` as teams ON wc.TeamId = teams.id
LEFT JOIN
    `DATASET.Lookup_Tables.Event_Types` as event_types ON Event_Id = event_types.Id
WHERE
Event = 'penaltyMissed' OR Event = 'keeperPenaltySaved' OR Event = 'penaltyScored' OR Period = 6
'''
penalties = pd.read_gbq(query,project_id='DATASET').drop('Event',axis=1).drop_duplicates()

In [ ]:
penalty_shots = penalties[penalties['Type'].isin(['Goal', 'MissedShots','SavedShot','ShotOnPost'])]

In [ ]:
x =(44.7, 55.4)
y = 26
x_adjust = [x[1] + 4, x[0] - 4]
x_bins = np.arange(x[0],x[1]+0.1,(x[1]-x[0])/6) #array([44.7 , 46.83, 48.96, 51.09, 53.22])
y_bins = np.arange(0,26+0.1,26/4) #array([ 0. ,  8.66666667, 17.33333333])

TEXTBOX = 'grey' 
SUCCESS_COLOR =  CONTESTED_COLOR
FAIL_COLOR = BAD_COLOR
fig,ax = plt.subplots(figsize=(10,8))
ax.set_xlim(x_adjust)
ax.set_ylim((-1,100))
ax.vlines(x,0,y,linewidth =1,color =NOTES_COLOR)
ax.plot(x_adjust,[0,0],linewidth = 1,color =NOTES_COLOR)
ax.plot(x,[y,y],linewidth = 1,color =NOTES_COLOR)


penalty_shots['X_bin'] = pd.cut(penalty_shots['GoalMouthY'],x_bins,labels = False)
penalty_shots['Y_bin'] = pd.cut(penalty_shots['GoalMouthZ'],y_bins,labels = False)
grouped = penalty_shots.groupby(['X_bin','Y_bin']).size().reset_index().rename(columns={0:'count'}).fillna(0)
arr = np.empty((len(y_bins)-1,len(x_bins)-1),dtype='float')
x_increment = (x[1]-x[0])/12
y_increment = 26/8
for i in range(len(x_bins)-1):
    for j in range(len(y_bins)-1):
        val = grouped[(grouped['X_bin']==i) & (grouped['Y_bin']==j)]['count'].values
        if len(val) == 0: 
            arr[j,i] = 0
            ax.text(x_bins[i]+x_increment,y_increment+y_bins[j],'0%',color='black',font=FONT,fontsize=9,ha='center',va='center',fontweight = 'demi',bbox = dict(facecolor=TEXTBOX,alpha = 0.2,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))
        else :
            arr[j,i] = val[0] / len(penalty_shots)
            ax.text(x_bins[i]+x_increment,y_increment+y_bins[j],f'{(val[0]/len(penalty_shots) * 100):.1f}%',color='black',font=FONT,fontsize=9,ha='center',va='center',fontweight = 'demi',bbox = dict(facecolor=TEXTBOX,alpha = 0.2,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))

success = penalty_shots[penalty_shots['Type']=='Goal']
fail = penalty_shots[penalty_shots['Type']!='Goal']
ax.scatter(success['GoalMouthY'],success['GoalMouthZ'],c=SUCCESS_COLOR,s=10)
ax.scatter(fail['GoalMouthY'],fail['GoalMouthZ'],c=FAIL_COLOR,s=10, marker = 'x')
    
ax.pcolormesh(x_bins,y_bins,arr,cmap='Blues',alpha=0.5)

ax.set_xlim(x_adjust)
ax.set_ylim((-1,80))
ax.axis('off')
fig.set_facecolor(BG_COLOR)


_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.87],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.81,'Nerves of Steel',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.78, f'''This visual highlights the locations of the {len(penalty_shots)} penalties were taken in the 2022 World Cup. 
Of which {len(penalty_shots[penalty_shots['Period'] == 6])} were taken in shootouts. There were a total of <{len(success)} successful> penalties and 
<{len(fail)} unsuccessful> penalties, with a total conversion rate of {len(success)/len(penalty_shots) * 100:.1f}%''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':SUCCESS_COLOR},{'color':FAIL_COLOR}])

fig.text(0.89,0.72,'@nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)
fig.text(0.89,0.098,'Data via Opta, up to Semifinals',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)



### 10. Corners
Here just show the successrate and long/short. Inswining outswining use a stacked bar chart

In [ ]:
flag = []
for i,row in all_touches_master.iterrows():
    curr = row['SatisfiedEventsTypes']
    if len([x for x in curr if ('corner' in x) or ('Corner' in x)]):
        flag.append(1)
    else:
        flag.append(0)
all_touches_master['CornerFlag'] = flag

In [ ]:
corner_idx= all_touches_master[all_touches_master['CornerFlag'] == 1].drop('SatisfiedEventsTypes',axis=1).drop_duplicates().index.tolist()
corners = all_touches_master.iloc[corner_idx].copy()

In [ ]:
def plot_one_corner(team,corners,ax):
    pitch = VerticalPitch(half=True,pitch_type='opta',pad_bottom=-20,pad_top=15,pitch_color=BG_COLOR,line_color=NOTES_COLOR,linewidth=0.6,line_alpha=0.8)
    ax.plot([0,100],[63.3,63.3],color=NOTES_COLOR,linewidth=0.6,alpha=0.8)
    pitch.draw(ax=ax)
    team_df = corners[corners['team']==team]
    success = team_df[team_df['OutcomeType'] == True]
    fail = team_df[team_df['OutcomeType'] == False]
    success_left = success[success['Y']>50]
    success_right = success[success['Y']<50]
    fail_left = fail[fail['Y']>50]
    fail_right = fail[fail['Y']<50]
    pitch.scatter(success_left['PassEndX'],success_left['PassEndY'],ax=ax,color=LEFT_COLOR,s=20,alpha=0.8,zorder=5,linewidths=0)
    pitch.scatter(success_right['PassEndX'],success_right['PassEndY'],ax=ax,color=RIGHT_COLOR,s=20,alpha=0.8,zorder=5,linewidths=0)
    pitch.scatter(fail_left['PassEndX'],fail_left['PassEndY'],ax=ax,color=LEFT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0,marker= "X")
    pitch.scatter(fail_right['PassEndX'],fail_right['PassEndY'],ax=ax,color=RIGHT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0,marker= "X")

    ax.text(70,102,team.upper(),fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')
    logoax = ax.inset_axes([0.8,0.82,0.12,0.12])
    im = plt.imread(f'{team}.png')
    logoax.imshow(im)
    logoax.axis('off')
    
    #stats
    tmp = fbref_teams[fbref_teams['Standard Squad'] == team][['Standard Squad','Pass Types Corner Kicks In','Pass Types Corner Kicks Out','Pass Types Corner Kicks Str']]
    inswing = tmp['Pass Types Corner Kicks In'].values[0]
    outswing = tmp['Pass Types Corner Kicks Out'].values[0]
    straight = tmp['Pass Types Corner Kicks Str'].values[0]

    bar_ax = ax.inset_axes([0.237,0.08,0.56,0.08])
    bar_ax.barh(0,inswing,color=IN_COLOR,height=0.2)
    if inswing > 0:
        bar_ax.text(inswing/2,0,f'{inswing:.0f}',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=7)
    if straight > 0:
        bar_ax.text(inswing + straight/2,0,f'{straight:.0f}',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=7)
    if outswing > 0:
        bar_ax.text(inswing + straight + outswing/2,0,f'{outswing:.0f}',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=7)
    bar_ax.barh(0,straight,color=STRAIGHT_COLOR,height=0.2,left = inswing)
    bar_ax.barh(0,outswing,color=OUT_COLOR,height=0.2,left = inswing + straight)
    bar_ax.axis('off')
    return ax



In [ ]:
def plot_legend_corner(ax):
    pitch = VerticalPitch(half=True,pitch_type='opta',pad_bottom=-20,pad_top=15,pitch_color=BG_COLOR,line_color=NOTES_COLOR,linewidth=0.6,line_alpha=0.4)
    ax.plot([0,100],[63.3,63.3],color=NOTES_COLOR,linewidth=0.6,alpha=0.4)
    pitch.draw(ax=ax)
    pitch.scatter(80,60,ax=ax,color=LEFT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0)
    pitch.scatter(80,40,ax=ax,color=LEFT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0,marker = "X")
    pitch.scatter(75,60,ax=ax,color=RIGHT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0)
    pitch.scatter(75,40,ax=ax,color=RIGHT_COLOR,s=50,alpha=0.8,zorder=5,linewidths=0,marker = "X")
    ax.text(50,102,'LEGEND',fontsize=9,ha='center',va='bottom',color = NOTES_COLOR,font=FONT,fontweight='demibold')

    ax.text(60,84,'Successful',fontsize=6,ha='center',va='center',color = NOTES_COLOR,font=FONT,bbox = dict(facecolor=TEXTBOX,alpha = 0.1,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))
    ax.text(40,84,'Unsuccessful',fontsize=6,ha='center',va='center',color = NOTES_COLOR,font=FONT,bbox = dict(facecolor=TEXTBOX,alpha = 0.1,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))
    ax.text(70,75,'Corners from right',fontsize=6,ha='right',va='center',color = RIGHT_COLOR,font=FONT,bbox = dict(facecolor=TEXTBOX,alpha = 0.1,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))
    ax.text(70,80,'Corners from left',fontsize=6,ha='right',va='center',color = LEFT_COLOR,font=FONT,bbox = dict(facecolor=TEXTBOX,alpha = 0.1,boxstyle = 'round, rounding_size = 0.2',edgecolor = TEXTBOX,pad = 0.2))

    bar_ax = ax.inset_axes([0.237,0.05,0.56,0.08])
    bar_ax.barh(0,33,color=IN_COLOR,height=0.2)
    bar_ax.barh(0,33,color=STRAIGHT_COLOR,height=0.2,left = 33)
    bar_ax.barh(0,33,color=OUT_COLOR,height=0.2,left = 66)
    bar_ax.axis('off')
    bar_ax.text(16.5,0,'Inswinging',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=6)
    bar_ax.text(50,0,'Straight',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=6)
    bar_ax.text(83.5,0,'Outswinging',ha='center',va='center',color=NOTES_COLOR,font=FONT,fontsize=6)
    return 


In [ ]:
LEFT_COLOR = BAD_COLOR
RIGHT_COLOR = GOOD_COLOR
STRAIGHT_COLOR = CONTESTED_COLOR
IN_COLOR = 'blue'
OUT_COLOR ='green'

fig,axs  = plt.subplots(7,5,figsize=(20,22))
fig.set_facecolor(BG_COLOR)
for ax,team in zip(axs.flatten()[:-3],teams):
    ax = plot_one_corner(team,corners,ax)
for ax in axs.flatten()[-3:]:
    ax.axis('off')
legend_ax = axs[0,4].inset_axes([0,1.15,1,1])
plot_legend_corner(legend_ax)

# fig.text(0.11,0.09,' ')
# fig.text(0.92,0.985,' ')
_ = fig.add_artist(lines.Line2D([0.12,0.9],[0.96],linewidth=1,color=TITLE_COLOR))
fig.text(0.125,0.937,'Corners',font=TITLE_FONT,color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='heavy')
fig_text(0.125,0.925, '''The prevalence of tournament football has increased the importance of set pieces of which the most dangerous are corners.The following 
plots show the distribution of corner kicks taken by each team in the tournament both from the <left> and <right>. FBref has classified the corners 
into 3 categories: <inswinging>, <straight> and <outswinging> and this classification system is used here. ''',
font=FONT,color = NOTES_COLOR,fontsize = 10,fontweight ='roman',highlight_textprops=[{'color':LEFT_COLOR},{'color':RIGHT_COLOR},{'color':IN_COLOR},{'color':STRAIGHT_COLOR},{'color':OUT_COLOR}])

fig.text(0.9,0.14,'''Data via Opta, Classification of Corner Data via FBRef''',
                ha='right',font = FONT,fontsize = 9, color = NOTES_COLOR, fontstyle ='italic',alpha=0.7)
fig.text(0.9,0.13,'Created by @nrehiew',ha='right',font = FONT,fontsize = 7, color = NOTES_COLOR,alpha=0.7)

plt.subplots_adjust(hspace=-0.1,wspace=0.2)

### 11. Player Dashboards 
- Musialia Take ons 

In [ ]:
import math
from matplotlib.patches import FancyBboxPatch, FancyArrow
from PIL import Image, ImageDraw

In [ ]:
flag = []
for i,row in all_touches_master.iterrows():
    curr = row['SatisfiedEventsTypes']
    if len([x for x in curr if ('keyPass' in x) or ('keyPass' in x)]):
        flag.append(1)
    else:
        flag.append(0)
all_touches_master['KeyPassFlag'] = flag

In [ ]:
all_touches_master[all_touches_master['KeyPassFlag'] == 1].drop('SatisfiedEventsTypes',axis=1).drop_duplicates().groupby('Player').size().sort_values(ascending=False).head(10)

In [ ]:
def create_carries(df):
    def process_df(df):
        df = df.sort_values(['MatchId', 'Period','Minute','Second']).drop(['Events','SatisfiedEventsTypes','Zone'],errors='ignore',axis=1).drop_duplicates().reset_index(drop=True)
        df['TotalTime'] = df['Minute']*60 + df['Second']
        passed_from = [None]
        for i,row in df.iloc[1:].iterrows():
            prev = df.loc[i - 1]
            if (prev['Type'] == 'Pass') & (prev['OutcomeType'] == True) & (prev['MatchId'] == row['MatchId']) & (abs(prev['TotalTime'] - row['TotalTime']) < 10):
                passed_from.append(prev['Player'])
            else:
                passed_from.append(None)

        passed_to  = []
        for i,row in df.iterrows():
            if (row['Type'] == 'Pass') & (row['OutcomeType'] == True) & (row['MatchId'] == row['MatchId']) & (abs(row['TotalTime'] - row['TotalTime']) < 10):
                passed_to.append(df.loc[i+1]['Player'])
            else: 
                passed_to.append(None)
        df['PassedFrom'] = passed_from
        df['PassedTo'] = passed_to
        return df
    processed = process_df(df)
    df_passes = processed[(~processed['PassEndX'].isna()) & (processed['OutcomeType'] == True)].copy()
    #xt by carries
    fromx = [None]
    fromy =[None]
    for i,row in processed.iloc[1:].iterrows():
        prev = processed.loc[i - 1]
        if (row['PassedFrom'] != None) & (prev['MatchId'] == row['MatchId']) & (prev['Period'] == row['Period']):
            fromx.append(prev['PassEndX'])
            fromy.append(prev['PassEndY'])
        else:
            fromx.append(None)
            fromy.append(None)
    processed['FromX'] = fromx
    processed['FromY'] = fromy
    processed = processed[processed['FromX'].notna()]
    return processed
processed = create_carries(all_touches_master)

In [ ]:
def create_sca_pass(df):
    raw = df.drop('SatisfiedEventsTypes',axis=1).drop_duplicates().sort_values(['MatchId','Period','Minute','Second']).reset_index(drop=True).copy()    
    sca = [False,False]
    raw = raw[raw['Type']=='Pass'].reset_index(drop=True)
    for i,row in raw.iloc[:-2].iterrows():
        next = raw.loc[i+1]
        next_s = raw.iloc[i+2]
        if ((next['Type'] in['MissedShots', 'SavedShot','ShotOnPost', 'Goal']) and (next['team'] == row['team'])  or  (next_s['Type'] in['MissedShots', 'SavedShot','ShotOnPost', 'Goal']) and (next_s['team'] == row['team'])) and (row['Type']=='Pass'):
            sca.append(True)
        else:
            sca.append(False)
    raw['sca_pass'] = sca
    return raw
sca_pass = create_sca_pass(all_touches_master)

In [ ]:
sca_pass

In [ ]:
def plot_progressive_passes(ax,df,player):
    player_df = df[(df['Player'] == player) & (df['Type'] =='Pass') & (df['OutcomeType'] == True)]
    player_df['Dist'] = player_df.apply(lambda x: math.sqrt((x['PassEndX'] - x['X'])**2 + (x['PassEndY'] - x['Y'])**2),axis=1)
    player_df['DistToGoal'] = player_df['PassEndX'] - player_df['X']
    progressive = player_df[(player_df['Dist'] > 10) & (player_df['DistToGoal'] > 5) ] #start from 40?
    # pitch =  VerticalPitch(pitch_type='opta',pitch_color = BG_COLOR,line_color=LINE,linewidth = 0.6, line_alpha=0.4,line_zorder=2)
    # pitch.draw(ax=ax)
    pitch.scatter(progressive['PassEndX'],progressive['PassEndY'],ax=ax,color=GOOD_COLOR,s=20,alpha=0.8,zorder=5,linewidths=0)
    pitch.lines(progressive['X'],progressive['Y'],progressive['PassEndX'],progressive['PassEndY'],ax=ax,color=GOOD_COLOR,linewidth=1,alpha=0.8,zorder=5)
    ax.text(0,100.5,'Progressive Passmap',ha='right',font=FONT,fontsize=6.5,color=NOTES_COLOR,alpha=0.7,va='bottom')

In [ ]:
def plot_sonar(ax,df,player,cmap_lst):
    def get_player_df(player_df,bins):
        angle = player_df['Angle']
        player_df['Bins'] = pd.DataFrame(pd.cut(angle,bins=bins,include_lowest=True))
        length = player_df.groupby('Bins').mean(numeric_only=True)['Length']
        counts = player_df['Bins'].value_counts()
        working_df = pd.concat([length,counts],axis=1).reset_index()
        working_df['Midpoint'] = working_df['index'].apply(lambda x:x.mid)
        working_df.drop('index',axis=1,inplace=True)
        working_df.columns = ['Mean','Count','Midpoint']
        working_df['Scaled Length'] = working_df['Mean']
        mean_x = player_df['X'].mean()
        mean_y = player_df['Y'].mean()
        return player_df, working_df.fillna(0), mean_x,mean_y
    df = df[(df['Player'] == player) & (df['Type'] =='Pass') & (df['OutcomeType'] == True)]
    player_df, working_df,x,y = get_player_df(df,18)
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256) 
    colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba(working_df['Count']/working_df['Count'].mean())
    newax = ax.inset_axes([0.12,0.18,0.65,0.65],projection='polar')
    newax.set_theta_zero_location("N")
    newax.bar(working_df['Midpoint'],working_df['Scaled Length'],color = colors,width=0.35)
    ax.set_xlim([-0.2,1.2])

    ax.axis('off')
    newax.set_facecolor(BG_COLOR)
    newax.set_rticks([])
    newax.set_yticklabels([])
    newax.set_xticklabels([])
    newax.grid(False)
    newax.spines['polar'].set_color(NOTES_COLOR)
    newax.set_ylim(0,30)

    # arrow = FancyArrow(1.1,0.25,0,0.2, width=0.01, color=NOTES_COLOR,zorder =2,head_width = 0.02,head_length=0.02,edgecolor=None)
    # ax.add_patch(arrow)
    for y in [0.48,0.52,0.56]:
        t = Polygon([[-0.18,y-0.02],[-0.15,y ],[-0.12,y -0.02]],facecolor = NOTES_COLOR,edgecolor = None,alpha=0.5)
        ax.add_artist(t)
    ax.text(-0.15,0.41,'Attacking\nDirection',ha='center',font=FONT,fontsize=5.5,color=NOTES_COLOR,alpha=0.7,va='bottom')
    ax.text(1.02,0.7,'Pass Sonar',ha='center',font=FONT,fontsize=6.5,color=NOTES_COLOR,alpha=0.7,va='bottom')
    return 
# fig,ax =plt.subplots()
# fig.set_facecolor(BG_COLOR)
# plot_sonar(ax,all_touches_open_play,'Jamal Musiala',cmap_lst)

In [ ]:
def plot_bars(ax,val,colors,textcolor):
    for i in range(1,11):
        if math.ceil(val/10) < i:
            color = '#F9F4F5'
            alpha = 0.05
        else:
            color = colors[i-1]
            alpha = 1
        rect = FancyBboxPatch(((i *1.2)/10,0.3),0.00007,0.7,boxstyle="Round,pad=0.02",color=color,alpha=alpha)
        ax.add_patch(rect)
    ax.set_ylim([0.3,0.7])
    ax.set_xlim(0,1.25)
    return ax

def plot_table(ax,df,textcolor,cmap_lst,title):
    cmap=LinearSegmentedColormap.from_list('test',cmap_lst, N=256)
    colors = plt.cm.ScalarMappable(cmap=cmap).to_rgba([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
    axs = []
    for i,col in zip(range(6),df.columns[1:]):
        y_coord = (i*(0.9/6) + 0.06 )
        height = (0.9/6) - 0.05
        tmp_ax = ax.inset_axes([0.2,y_coord,0.68,height + 0.02])
        tmp_ax = plot_bars(tmp_ax,int(df[col] * 100),colors,textcolor)
        ax.text(0.843,y_coord + height/2,int(df[col] * 100),va='center',ha='right',color=textcolor,font=FONT,fontsize=7)
        ax.text(0.22,y_coord + height/2,col.replace('_',' '),va='center',ha='right',color=textcolor,font=FONT,fontsize=7)
        ax.text(0.85,y_coord + height/2 + 0.02,'th',fontsize=6,va='center',ha='left',color=textcolor,font=FONT)
        tmp_ax.axis('off')
        axs.append(tmp_ax)
    ax.set_facecolor(BG_COLOR)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['bottom'].set_color(NOTES_COLOR)
    ax.spines['top'].set_color(NOTES_COLOR) 
    ax.spines['right'].set_color(NOTES_COLOR)
    ax.spines['left'].set_color(NOTES_COLOR)
    ax.spines['bottom'].set_linewidth(0.4)
    ax.spines['top'].set_linewidth(0.4)
    ax.spines['right'].set_linewidth(0.4)
    ax.spines['left'].set_linewidth(0.4)
    ax.text(0.9,1.02,title,ha='right',font=FONT,fontsize=6.5,color=NOTES_COLOR,alpha=0.7,va='bottom')
    ax.set_xlim([0,0.9])
    ax.set_ylim(0,1)
    return ax,axs
tmp = fbref_players[(fbref_players['Standard Pos'].str.contains('MF') | fbref_players['Standard Pos'].str.contains('FW')) & (fbref_players['Standard Playing Time Min'] > 120)][['Standard Player','Goal And Shot Creation Sca Sca90','Possession Dribbles Att','Possession Dribbles Succ%','Passing Prog','Shooting Standard Sh/90','Standard Expected Npxg']]
tmp.columns = ['Player','SCA90','Dribbles Attempted','Dribbles Successful','Progressive Passes','Shots On Target/90','Expected Goals']
tmp[['SCA90','Dribbles Attempted','Dribbles Successful','Progressive Passes','Shots On Target/90','Expected Goals']] = tmp[['SCA90','Dribbles Attempted','Dribbles Successful','Progressive Passes','Shots On Target/90','Expected Goals']].rank(pct= True)
# fig,ax = plt.subplots()
# fig.set_facecolor(BG_COLOR)
# plot_table(ax, tmp[tmp['Player'] == 'Jamal Musiala'], NOTES_COLOR,cmap_lst,'Compared to Forwards with at least 120 mins played')

In [ ]:
def shotmap(df,ax,player,cmap_lst):
    df = df[df['Player'] == player]
    df = df[(df['Type'].isin(['MissedShots','ShotOnPost','SavedShot', 'Goal'])) & (df['Period'] != 6)].drop('SatisfiedEventsTypes',axis=1).drop_duplicates()
    pitch = VerticalPitch(half=True,pitch_type='opta',pitch_color = BG_COLOR,line_color='white',linewidth = 0.6, line_alpha=0.4,pad_bottom=0.5)
    pitch.draw(ax=ax)
    pitch.scatter(df['X'],df['Y'],ax=ax,color=df['Type'].apply(lambda x: cmap_lst[2] if x =='Goal' else 'gray'),s= 30 ,alpha = df['Type'].apply(lambda x: 1 if x =='Goal' else 0.4 ),linewidths=0,zorder =3)

    fontsize = 6.5
    fontweight = 'heavy'
    alpha = 0.75
    ax.text(90,70,'Goals:',bbox = dict(facecolor=cmap_lst[2],alpha=0.8,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha )
    ax.text(78,70,len(df[df['Type'] =='Goal']),fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha+0.25)
    ax.text(90,66,'Shots:',bbox = dict(facecolor='grey',alpha=alpha-0.2,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(78,66,len(df),fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(90,62,'xG/Shot:',bbox = dict(facecolor='grey',alpha=alpha-0.2,boxstyle = 'round'), fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    ax.text(77,62,fbref_players[fbref_players['Standard Player'] == player]['Shooting Expected Npxg/Sh'].values[0],fontsize=fontsize,ha='left',color = 'white',font=FONT,fontweight=fontweight,alpha = alpha)
    return ax

In [ ]:
def heatmap_positional(df,ax,player,cmap_lst):
    player_df = all_touches_open_play[all_touches_open_play['Player'] == player]
    pitch = Pitch(pitch_type='opta',pitch_color = BG_COLOR,line_color=NOTES_COLOR,linewidth = 0.6, line_alpha=0.4,positional=True,goal_type='box')
    pitch.draw(ax=ax)
    bin_statistic = pitch.bin_statistic_positional(player_df['X'],player_df['Y'], statistic='count',positional='full', normalize=True)
    cmap = LinearSegmentedColormap.from_list('mycmap', cmap_lst)
    pitch.heatmap_positional(bin_statistic, ax=ax,  cmap=cmap, edgecolors=NOTES_COLOR,linewidth = 0,alpha = 0.35)
    labels = pitch.label_heatmap(bin_statistic, color='#f4edf0', fontsize=9, ax=ax, ha='center', va='center',str_format='{:.0%}')
    ax.text(99.5,101.5,'Touchmap by zone',ha='right',font=FONT,fontsize=6.5,color=NOTES_COLOR,alpha=0.7,va='bottom')
    return ax


In [ ]:
xt

In [ ]:
def create_dashboard(style,player):
    fig,ax  = plt.subplots(figsize=(15,12))
    fig.set_facecolor(BG_COLOR)
    pitch = VerticalPitch(pitch_type='opta',pitch_color = BG_COLOR,line_color=LINE,linewidth = 0.5, line_alpha=0.6,line_zorder=2,spot_scale=0.003)
    pitch.draw(ax=ax)
    if style == 1:
        ax1 = ax.inset_axes([1.02,-0.09,0.65,0.65])
        ax2 = ax.inset_axes([1.05,0.28,0.65,0.6])
        ax3 = ax.inset_axes([1.05,0.773,0.62,0.2])
    elif style == 2:
        ax1 = ax.inset_axes([1.02,-0.09,0.65,0.65])
        ax2 = ax.inset_axes([1.02,0.28,0.65,0.65])
        ax3 = ax.inset_axes([1.05,0.773,0.62,0.2])
    elif style == 3:
        ax1 = ax.inset_axes([1.02,-0.09,0.65,0.65])
        ax2 = ax.inset_axes([1.02,0.28,0.65,0.65])
        ax3 = ax.inset_axes([1.02,0.63,0.65,0.65])
    _ = fig.add_artist(lines.Line2D([0.42,0.7],[0.935],linewidth=4,color=TITLE_COLOR))
    fig.text(0.42,0.95,player,font='Assistant',color = NOTES_COLOR,fontsize = TITLE_SIZE,fontweight ='bold')   
    fig.text(0.98,0.13,'Inspired by: @jonollington  |  Created by @nrehiew',color = NOTES_COLOR,fontsize = 6.2,font = FONT,alpha=0.6,ha='right')
    minutes = fbref_players[fbref_players['Standard Player'] == player]['Standard Playing Time Min'].values[0]
    age = fbref_players[fbref_players['Standard Player'] == player]['Standard Age'].values[0].split('-')[0]
    club = fbref_players[fbref_players['Standard Player'] == player]['Standard Club'].values[0].split(' ',maxsplit = 1)[1]
    im_ax = ax.inset_axes([0.05,0.83,0.2,0.5])
    team = fbref_players[fbref_players['Standard Player'] == player]['Standard Squad'].values[0]
    print(team)
    fname = rf'{team}.png'
    a = plt.imread(fname)
    im_ax.imshow(a)
    im_ax.axis('off')
    fig.text(0.42,0.91,f"{minutes} played  |  {age} years  |  Plays for: {club}",font='Assistant',color = NOTES_COLOR,fontsize = 14)  
    fig.text(0.5,0.98,' ')
    return fig, [ax,ax1,ax2,ax3]

player = 'Antoine Griezmann'
fig, axs = create_dashboard(1,player)
plot_progressive_passes(axs[0],all_touches_open_play,player)
#shotmap(all_touches_open_play,axs[1],'Jamal Musiala',cmap_lst)
heatmap_positional(all_touches_open_play,axs[1],player,cmap_lst)
plot_sonar(axs[2],all_touches_open_play,player,cmap_lst)
plot_table(axs[3], tmp[tmp['Player'] == player], NOTES_COLOR,cmap_lst,'Percentile Rank Compared to Forwards with at least 120 mins played')